## Handling large datasets
Pandas for large datasets -
https://www.dataquest.io/blog/pandas-big-data/

In [32]:
import pandas as pd
import numpy as np

In [37]:
btomb = 1024**2

In [4]:
df = pd.read_csv("/home/course/public/Datasets/crime.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
usage = df.memory_usage()
usage = usage / 1024 ** 2
usage 

Index                    0.000122
ID                      54.020676
Case Number             54.020676
Date                    54.020676
Block                   54.020676
IUCR                    54.020676
Primary Type            54.020676
Description             54.020676
Location Description    54.020676
Arrest                   6.752584
Domestic                 6.752584
Beat                    54.020676
District                54.020676
Ward                    54.020676
Community Area          54.020676
FBI Code                54.020676
X Coordinate            54.020676
Y Coordinate            54.020676
Year                    54.020676
Updated On              54.020676
Latitude                54.020676
Longitude               54.020676
Location                54.020676
dtype: float64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7080598 entries, 0 to 7080597
Data columns (total 22 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Case Number           object 
 2   Date                  object 
 3   Block                 object 
 4   IUCR                  object 
 5   Primary Type          object 
 6   Description           object 
 7   Location Description  object 
 8   Arrest                bool   
 9   Domestic              bool   
 10  Beat                  int64  
 11  District              float64
 12  Ward                  float64
 13  Community Area        float64
 14  FBI Code              object 
 15  X Coordinate          float64
 16  Y Coordinate          float64
 17  Year                  int64  
 18  Updated On            object 
 19  Latitude              float64
 20  Longitude             float64
 21  Location              object 
dtypes: bool(2), float64(7), int64(3), object(1

In [45]:
print(np.iinfo('uint32'), np.iinfo('uint16'), df.ID.max())

Machine parameters for uint32
---------------------------------------------------------------
min = 0
max = 4294967295
---------------------------------------------------------------
 Machine parameters for uint16
---------------------------------------------------------------
min = 0
max = 65535
---------------------------------------------------------------
 11997836


In [43]:
#conv_ID = df.ID.apply(pd.to_numeric,downcast='unsigned')
conv_df = pd.DataFrame()
conv_df['ID'] = df.ID.astype("uint32")

In [62]:
print(df[['ID']].memory_usage() / btomb)
print(conv_df.memory_usage() / btomb)

Index        0.000122
ID          54.020676
District    54.020676
dtype: float64
Index        0.000122
ID          27.010338
District     6.752584
dtype: float64


In [50]:
print(df.District.min(), df.District.max())

1.0 31.0


In [51]:
conv_df['District'] = df.District.astype("uint8")

ValueError: Cannot convert non-finite values (NA or inf) to integer

There are NaNs. So first substitute them with an appropriat integer that will be possible to filter out later. 0 seems to be a good choice fo this

In [53]:
 df.loc[df.District.isna(), 'District'] = 0

In [54]:
conv_df['District'] = df.District.astype("uint8")

In [55]:
conv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7080598 entries, 0 to 7080597
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   ID        uint32
 1   District  uint8 
dtypes: uint32(1), uint8(1)
memory usage: 33.8 MB


In [63]:
print(df[['ID', 'District']].memory_usage() / btomb)
print(conv_df.memory_usage() / btomb)

Index        0.000122
ID          54.020676
District    54.020676
dtype: float64
Index        0.000122
ID          27.010338
District     6.752584
dtype: float64


In [61]:
sim = conv_df['District'].astype("float") != df.District
print("Nr. of discrepancies: %d"%sim.sum())

Nr. of discrepancies: 0


In [65]:
df.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [74]:
print(df['X Coordinate'][df['X Coordinate'].notna()].min(), df['X Coordinate'][df['X Coordinate'].notna()].max())

0.0 1205119.0


In [78]:
np.finfo('float16')

finfo(resolution=0.001, min=-6.55040e+04, max=6.55040e+04, dtype=float16)

In [82]:
(df['X Coordinate'][df['X Coordinate'].notna()].astype('int').astype('float64') != df['X Coordinate'][df['X Coordinate'].notna()]).sum()

0

Seems like none of the floats are really float

### Objects are still there. What can we do with them?

We could use categories where it is appropriate. If the number of unique elements are very low, let's say less then 100 and (we can name them easily), then it really makes sense.

In [86]:
print(len(df['Primary Type'].unique()), df['Primary Type'].notna().sum())

35 7080598


Looks like 'Primary type' is a column where categories would make sense.

In [88]:
conv_df['Primary Type'] = df['Primary Type'].astype('category')

In [90]:
print(df[['ID', 'District', 'Primary Type']].memory_usage() / btomb)
print(conv_df.memory_usage() / btomb)

Index            0.000122
ID              54.020676
District        54.020676
Primary Type    54.020676
dtype: float64
Index            0.000122
ID              27.010338
District         6.752584
Primary Type     6.754072
dtype: float64


In [91]:
conv_df.head()

,ID,District,Primary Type
0,11034701,4,DECEPTIVE PRACTICE
1,11227287,22,CRIM SEXUAL ASSAULT
2,11227583,8,BURGLARY
3,11227293,3,THEFT
4,11227634,1,CRIM SEXUAL ASSAULT


Caveat!

In [92]:
ages = np.array([10, 15, 13, 12, 23, 25, 28, 59, 60])
pd.cut(ages, bins=5)

There are codes referring to the category

In [96]:
conv_df['Primary Type'].cat.codes

0           8
1           5
2           3
3          33
4           5
           ..
7080593    25
7080594    11
7080595    11
7080596     8
7080597     8
Length: 7080598, dtype: int8

In [99]:
df[['Date']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7080598 entries, 0 to 7080597
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Date    object
dtypes: object(1)
memory usage: 54.0+ MB


In [100]:
df[['Date']].head()

,Date
0,01/01/2001 11:00:00 AM
1,10/08/2017 03:00:00 AM
2,03/28/2017 02:00:00 PM
3,09/09/2017 08:17:00 PM
4,08/26/2017 10:00:00 AM


In [205]:
#pd.to_datetime(df[['Date']],format='%m/%d/%Y %I:%M:%S %p', errors='raise')
conv_df['Date'] = df[['Date']].apply(lambda x: pd.to_datetime(x, format='%m/%d/%Y %I:%M:%S %p'))
#pd.to_datetime(df[['Date']].iloc[:10])

#df["DateTime"] = pd.to_datetime(df["DateTime"],errors="coerce").dt.strftime("%d-%m-%Y %H:%M:%S")

In [206]:
print(df[['ID', 'District', 'Primary Type', 'Date']].memory_usage() / btomb)
print(conv_df.memory_usage() / btomb)

Index            0.000122
ID              54.020676
District        54.020676
Primary Type    54.020676
Date            54.020676
dtype: float64
Index            0.000122
ID              27.010338
District         6.752584
Primary Type     6.754072
Date            54.020676
dtype: float64


In [207]:
conv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7080598 entries, 0 to 7080597
Data columns (total 4 columns):
 #   Column        Dtype         
---  ------        -----         
 0   ID            uint32        
 1   District      uint8         
 2   Primary Type  category      
 3   Date          datetime64[ns]
dtypes: category(1), datetime64[ns](1), uint32(1), uint8(1)
memory usage: 94.5 MB


In [220]:
cs = conv_df[['ID', 'District', 'Primary Type']].memory_usage().sum()
ds = df[['ID', 'District', 'Primary Type']].memory_usage().sum()
print("%d %%"%((cs/ds)*100))


25 %
